# Spatial and Spectral Tile

作者：于峻川 

邮箱：jasonyu@live.cn






idl对于大影像处理时需要进行分块操作，也是idl进行遥感应用的核心技术，通过该测试了解idl空间分块和光谱分块的过程。

In [0]:
  input_file=envi_pickfile(/multiple_files,title='Pick the input file for testing')
  out_dir=envi_pickfile(/directory,title='pick output directory')
  out_fname=out_dir+'\'+(strsplit((file_basename(input_files)),'.',/extract))[0]+'_output.img'
  
  ENVI_OPEN_FILE,inputfile,R_FID=fid
  envi_file_query, fid, dims=dims, nb=nb,NS=ns,NL=nl,data_type=data_type，interleave=interleave
  mapinfo=envi_get_map_info(fid=fid) 
  pos=indgen(nb)；

通过上面的语句定义输入文件和输出文件名，并读取相关参数，主要包括dims，nb，ns，nl，data_type，interleave,pos以及mapinfo。测试使用的数据ENVI自带的can_tmr.img其获取参数如下所示：

In [1]:
dims=[-1,0,639,0,399]
ns=640
nl=640
nb=6
data_type=1
interleave=0
pos=[0,1,2,3,4,5]  ; 

接下来是实现idl的空间分块的操作，首先建立一个空文件用来保存结果文件

In [2]:
  openw,lun, out_fname,/get_lun；

分块用的是envi_init_tile命令，该命令利用interleave的值来确定进行空间分块还是光谱分块，0表示空间分块，1或2表示光谱分块，因此如果使用空间分块可以直接将interleave赋值为0，如果使用光谱分块可以如果使用光谱分块可以使用interleave=interleave>1来实现，该语句表明如果输入文件为baq或bil格式（interleave均≤1），interleave值将赋予1，如果输入文件为bip格式，interleave值将赋予2.

In [3]:
 tile_id = envi_init_tile(fid, pos, interleave=0,  num_tiles=num_tiles,$
 xs=dims[1], xe=dims[2],  ys=dims[3], ye=dims[4])；

无论数据格式如何，空间分块的大小是不变的，均为ns×nl,空间分块的个数为nb，光谱分块的大小由输入格式决定，如果为bil，则分块大小为ns×nb，如果为bip，则分块大小为nb×ns，光谱分块个数不变为nl。
由于空间分块中num_tiles等于波段数，因此适合以波段为计算单位的操作。光谱分块无法进行波段运算的过程进行分块，只适用于进行光谱计算相关的操作。

In [4]:
for i=0L, num_tiles-1 do begin
    data= envi_get_tile(tile_id, i)
    writeu,lun,data
endfor
  
  free_lun, lun
  close,lun
  envi_tile_done, tile_id
  envi_setup_head, fname=out_fname, INTERLEAVE=0, ns=ns, nl=nl,  nb=nb,$
  data_type=data_type,  map_info=map_info, /write, /open；

如前所述，data为空间分块后获取的一个tile，它的维度是640×400。需要注意的是，对于基于波段的具体计算过程均要在for循环中表述，该计算过程也必须针对单一tile进行，程序会利用for循环将计算过程应用到所有的tile。在这种直接由envi_get_tile赋值data的前提下，writeu语句也必须写入for循环之内，才能保证输出data是具有nb个波段的数据。此时，如果将writeu放在for循环的外侧，必须重新对data进行定义，否则输入结果data的维度只是一个tile的大小（最后一个tile，nb个数为1），而envi_setup_head中已写明数据波段数为nb，这种情况下打开结果文件会报错：

In [11]:
Warning: The dimensions and data type of the given file are inconsistent with the actual size of the file.
Please Correct.；

如果需要想将writeu写在循环的外面，必须重新定义data，使其能够按照nb进行循环，并在循环后变成一个完整的具有nb个波段的数据立方体，具体写法如下：

In [5]:
 tile_id = envi_init_tile(fid, pos, interleave=0,  num_tiles=num_tiles,$
 xs=dims[1], xe=dims[2],  ys=dims[3], ye=dims[4])
 data=bytarr(ns,nl,nb)
  for i=0L, num_tiles-1 do begin
    data[*,*,i]= envi_get_tile(tile_id, i)
  endfor
  
  writeu,lun,data
  free_lun, lun
  close,lun
  envi_tile_done, tile_id
  envi_setup_head, fname=out_fname, INTERLEAVE=0, ns=ns, nl=nl,  nb=nb,$
  data_type=data_type,  map_info=map_info, /write, /open；

事实上通过测试表明，在进行空间分块的时候，输入文件的格式（bsq/bil/bip）并不重要，只要定义函数envi_init_file中的interleave的值为0，函数将执行空间分块，且输出文件必须定义为bsq格式，即interleave=0.而对于光谱分块来说，情况更为复杂一些，首先，光谱分块的输入文件不可为bsq格式，其次，输入文件与输出文件格式需一致。因此，对于光谱分块来说envi_setup_dead中，interleave=interleave。为了方便操作可以在for循环中加入一个if语句将bip格式转换为bil格式，这样输出文件直接定义为interleave=1即可。

In [9]:
  if (interleave eq 2) then begin
    data=transpose(data)
  endif ；

下面给出一段完整的光谱分块的程序：

In [7]:
PRO spectral_tile_TEST
  compile_opt strictarr
  envi, /restore_base_save_files
  ENVI_BATCH_INIT
  
  inputfile='C:\Users\JASONYU.JASONYU-PC\Desktop\test.img'
  out_fname='C:\Users\JASONYU.JASONYU-PC\Desktop\test_out.img'
  
  ENVI_OPEN_FILE,inputfile,R_FID=fid
  envi_file_query, fid, dims=dims, nb=nb,NS=ns,NL=nl,data_type=data_type,interleave=interleave
  mapinfo=envi_get_map_info(fid=fid)
  
  pos=indgen(nb)
  openw,lun, out_fname,/get_lun
  tile_id = envi_init_tile(fid, pos, interleave=interleave>1,  num_tiles=num_tiles,$
    xs=dims[1], xe=dims[2],  ys=dims[3], ye=dims[4])
  for i=0L, num_tiles-1 do begin
    data= envi_get_tile(tile_id, i)
    if (interleave eq 2) then begin
      data=transpose(data)
    endif
    writeu,lun,data
  endfor
  free_lun, lun
  close,lun
  envi_tile_done, tile_id
  envi_setup_head, fname=out_fname, INTERLEAVE=1, ns=ns, nl=nl,nb=nb, data_type=data_type,$
  map_info=map_info, /write, /open
END；